In [1]:
from pyMSOO.MFEA.model import SBSGA
from pyMSOO.utils.Crossover import *
from pyMSOO.utils.Mutation import *
from pyMSOO.utils.Selection import *
from pyMSOO.utils.DimensionAwareStrategy import *
from pyMSOO.MFEA.benchmark.continous import *
from pyMSOO.utils.MultiRun.RunMultiTime import * 

In [3]:
# tasks, IndClass = CEC17_benchmark.get_2tasks_benchmark(1)
# tasks, IndClass = WCCI22_benchmark.get_complex_benchmark(1)
tasks, IndClass = CEC17_benchmark.get_10tasks_benchmark()

In [4]:
baseModel = SBsSGA.model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    dimension_strategy=DaS_strategy(eta= 3)
)
solve = baseModel.fit(
    nb_generations = 1000, rmp = 0.5, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

Seed: None -- Time: 01m 34.59s  100 % [====================>]  Cost: 3.50E-06  5.89E-07  1.39E-16  0.00E+00  4.68E-04  1.90E-04  5.47E-07  6.38E-04  6.84E-14  3.98E+00  ,  
END!


### MUltitime Model

In [26]:
baseModel = MultiTimeModel(model= model)
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    dimension_strategy= NoDaS(),
)
solve = baseModel.fit(
    nb_generations = 1000,nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

In [27]:
baseModel.run(
    nb_run= 30,
    # save_path= './RESULTS/MFEA_cec17.mso'
)

Seed: 0 -- Time: 01m 21.29s  100 % [====================>]  Cost: 1.92E-06  2.73E-07  1.79E-10  0.00E+00  2.59E-04  1.66E-04  1.68E-03  6.39E-04  3.48E-08  1.67E+01  ,  
END!
Seed: 1 -- Time: 01m 17.42s  100 % [====================>]  Cost: 1.91E-06  2.89E-07  1.78E-08  0.00E+00  2.51E-04  1.72E-04  7.43E-03  6.38E-04  4.25E-08  2.11E+01  ,  
END!
Seed: 2 -- Time: 01m 18.05s  100 % [====================>]  Cost: 1.10E-06  2.70E-07  1.96E-10  0.00E+00  1.64E-04  1.48E-04  2.02E-03  6.38E-04  2.52E-08  3.47E+01  ,  
END!
Seed: 3 -- Time: 01m 21.33s  100 % [====================>]  Cost: 2.84E-06  3.02E-07  1.18E-10  0.00E+00  4.72E-04  1.69E-04  1.61E-03  6.38E-04  4.23E-08  2.30E+01  ,  
END!
Seed: 4 -- Time: 01m 13.80s   92 % [==================> ]  Cost: 9.49E-06  1.69E-06  4.16E-09  0.00E+00  1.12E-03  3.76E-04  3.97E-03  6.49E-04  9.89E-08  2.15E+01  ,  

f:\BTVN\DSAI\Optimization Lab\pyMSOO\pyMSOO\utils\MultiRun\RunMultiTime.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.array(result[:][:min([len(his) for his in result])][:])


functions : <generator object stream_list at 0x0000015A6ECA8900>, <class 'generator'> cannot saved in process_save_dict function
attributes : <generator object stream_list at 0x0000015A6ECA8970>, <class 'generator'> cannot saved in process_save_dict function
globals : <generator object stream_list at 0x0000015A6ECA89E0>, <class 'generator'> cannot saved in process_save_dict function
functions : <generator object stream_list at 0x0000015A6ECA8A50>, <class 'generator'> cannot saved in process_save_dict function
attributes : <generator object stream_list at 0x0000015A6ECA8AC0>, <class 'generator'> cannot saved in process_save_dict function
globals : <generator object stream_list at 0x0000015A6ECA8B30>, <class 'generator'> cannot saved in process_save_dict function
functions : <generator object stream_list at 0x0000015A6ECA8BA0>, <class 'generator'> cannot saved in process_save_dict function
attributes : <generator object stream_list at 0x0000015A6ECA8C10>, <class 'generator'> cannot saved

: 